## importation and routings

In [1]:
%cd '/UltimeTradingBot/Crypto_backtest_tools'
WINDOW_SIZE=5
BUY_PERCENT=0.28
MAX_FORCAST_SIZE=1
VERSION=1
TESTING_MOD=True
RETRAIN=True
JUST_IMPORT_DATA=True
#Normalization_File='w15_NoVol_Normalization.json'
#Model_FileName='w15_NoVol_XcryptoAi_model.hdf5'
ALLHIST_FILE='Results_history.json'
DATA_DIR='/UltimeTradingBot/Data/'
FIRST_NORM_FLAG=True
DATA_DIR='/UltimeTradingBot/Data'
Normalization_File=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Norm_v{VERSION}.json'
Model_FileName=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Model_v{VERSION}.hdf5'
DATA_FILE=f'{DATA_DIR}/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Data_v{VERSION}.csv'
#REMOTE_DATA_FILE=f'/gdrive/CSV/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Data_v{VERSION}.csv'
REMOTE_DATA_FILE=f'/gdrive/+DATA+/tp{int(BUY_PERCENT*100)}_w{WINDOW_SIZE}_max{MAX_FORCAST_SIZE}min_Data_v{VERSION}.csv'
window=WINDOW_SIZE
SELL_PERCENT=0.30
NORM_FILE=Normalization_File
MODEL_FILE=Model_FileName
Px=5
BUFFER_SIZE=200000*Px
SAMPLE_SIZE=4000*Px
NROWS=4000000
SKIPROWS=0

#DATA_FILE=DATA_DIR+'w'+str(WINDOW_SIZE)+'_EXTData.csv'



/UltimeTradingBot/Crypto_backtest_tools


# Fonctions and Data

In [2]:

import sys
sys.path.append('/UltimeTradingBot/Crypto_backtest_tools')
%cd /UltimeTradingBot/Crypto_backtest_tools
from utilities.get_data import get_historical_from_db
from utilities.backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics
import pandas as pd
import ccxt
import time
import matplotlib.pyplot as plt
#import ta
import numpy as np
import gc
import random
from sklearn.utils import shuffle
import seaborn as sns
import tensorflow as tf

from utilities.backtesting import plot_wallet_vs_asset, get_metrics, get_n_columns, basic_multi_asset_backtest, plot_sharpe_evolution, plot_bar_by_month
#from utilities.custom_indicators import SuperTrend
pd.options.mode.chained_assignment = None  # default='warn'
import gc
gc.collect()    
import ccxt
import matplotlib.pyplot as plt
import json
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 


import warnings
warnings.filterwarnings('ignore')
PRERR=False
def prerr(err):
    if PRERR:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

PDEBUG=True
def pdebug(err):
    if PDEBUG:
        print("\033[0;31m Error in "+str(sys._getframe().f_code.co_name) +" \033[0;33m"+str(err))

/UltimeTradingBot/Crypto_backtest_tools


2022-10-07 13:53:10.895158: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-07 13:53:11.055283: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Usefull Global and Config Vars

### importing Data

In [ ]:
Binance_USDT_HALAL=[
    "BTC/USDT",
    "ENS/USDT",
    "HIGH/USDT",
    "RAD/USDT",
    "XRP/USDT",
    "LUNA/USDT",
    "APE/USDT",
    "NEAR/USDT",
    "LTC/USDT",
    "ETC/USDT",
    "SHIB/USDT",
    "EGLD/USDT",
    "ALGO/USDT",
    "ETH/USDT",
    "DOGE/USDT",
    "SOL/USDT",
    "DOT/USDT",
    "AVAX/USDT",
    "ADA/USDT",
    "TRX/USDT"
]

pair_list = Binance_USDT_HALAL
tf = '1m'
oldest_pair = "BTC/USDT"
df_list1m = {}
df_list1d = {}
df_list1h = {}
df_list5m = {}
df_list15m = {}


for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1m', path="./database/")
    df_list1m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1d', path="./database/")
    df_list1d[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '1h', path="./database/")
    df_list1h[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(ccxt.binance(), pair, '5m', path="./database/")
    df_list5m[pair] = df.loc[:]

for pair in pair_list:
    df = get_historical_from_db(
        ccxt.binance(), pair, '15m', path="./database/")
    df_list15m[pair] = df.loc[:]
del(df)
df_list = df_list1m
prerr("Data load 100% use df_list1d[\"BTC/USDT\"] for exemple to access")


In [ ]:
### Testing Raw Data

In [ ]:
#df_list1m["BTC/USDT"].iloc[0:10]

### Validated Fonctions

In [ ]:
'''
Binance First Candle Finders
Creslin

Get list of all IDs on binance
Returns the first candle / launch timestamp to the minute for each
'''
import urllib.request
import json
import ccxt

def all_ids():
    # load all markets from binance into a list
    id = 'binance'
    exchange_found = id in ccxt.exchanges
    if exchange_found:
        exchange = getattr(ccxt, id)({})
        markets = exchange.load_markets()
        tuples = list(ccxt.Exchange.keysort(markets).items())

        ids = []
        for (k, v) in tuples:
            ids.append(v['id'])

        return ids

def give_first_kline_open_stamp(interval, symbol, start_ts=1499990400000):
        '''
        Returns the first kline from an interval and start timestamp and symbol
        :param interval:  1w, 1d, 1m etc - the bar length to query
        :param symbol:    BTCUSDT or LTCBTC etc
        :param start_ts:  Timestamp in miliseconds to start the query from
        :return:          The first open candle timestamp
        '''

        url_stub = "http://api.binance.com/api/v1/klines?interval="

        #/api/v1/klines?interval=1m&startTime=1536349500000&symbol=ETCBNB
        addInterval   = url_stub     + str(interval) + "&"
        addStarttime  = addInterval   + "startTime="  + str(start_ts) + "&"
        addSymbol     = addStarttime + "symbol="     + str(symbol)
        url_to_get = addSymbol

        kline_data = urllib.request.urlopen(url_to_get).read().decode("utf-8")
        kline_data = json.loads(kline_data)

        return kline_data[0][0]


# Get list of all IDs on binance
def get_crypto_metadata(pair_list):
    Binance_USDT_HALAL=pair_list
    ids = []
    #ids = all_ids()
    for halalpair in Binance_USDT_HALAL:
    #    print( halalpair.replace('/',''))
        ids.append(halalpair.replace('/',''))
    #print(ids)
    MetaData=pd.DataFrame(ids)
    MetaData["Pair"]=Binance_USDT_HALAL
    counters=0
    for this_id in ids:
        '''
        Find launch Week of symbol, start at Binance launch date 2017-07-14 (1499990400000)
        Find launch Day of symbol in week
        Find launch minute of symbol in day
        '''

        symbol_launch_week_stamp   = give_first_kline_open_stamp('1w', this_id, 1499990400000 )
        symbol_launch_day_stamp    = give_first_kline_open_stamp('1d', this_id, symbol_launch_week_stamp)
        symbol_launch_minute_stamp = give_first_kline_open_stamp('1m', this_id, symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_week_stamp"]=str(symbol_launch_week_stamp)
        MetaData.loc[counters,"launch_day_stamp"]=str(symbol_launch_day_stamp)
        MetaData.loc[counters,"launch_minute"]=pd.to_datetime(symbol_launch_minute_stamp, unit='ms')

        counters += 1

        #print("Week stamp", symbol_launch_week_stamp)
        #print("Day  stamp", symbol_launch_day_stamp)
        #print("Min  stamp", symbol_launch_minute_stamp)

        print(this_id, "launched", symbol_launch_minute_stamp )
    return MetaData
    #print("")

In [ ]:
try:
    MetaData = pd.read_csv("../Data/MetaData.csv",index_col=0)
except:
    MetaData=get_crypto_metadata(Binance_USDT_HALAL)
    MetaData.to_csv("../Data/MetaData.csv")
#allok = pd.read_csv('D:/+DATA+/allok_w15.csv')

In [ ]:
#MetaData.to_csv("D:\+DATA+\MetaData.csv")
pair_list=Binance_USDT_HALAL
window=WINDOW_SIZE
buy_weight=50
sample_size=10000
min_days=MAX_FORCAST_SIZE
buffer_size=100000
#MetaData

In [ ]:
def expand_row(dataframe, window=2):
    df = dataframe.copy()
    for i in range(1, window+1):
        df["high"+str(i)] = df["high"][i:]
        df["low"+str(i)] = df["low"][i:]
        df["open"+str(i)] = df["open"][i:]
        df["close"+str(i)] = df["close"][i:]
        df["volume"+str(i)] = df["volume"][i:]
    return df

def justlast_remover(df):
    justlast=["BTC_open","BTC_low","BTC_close","open","low","close"]
    for key in df.keys():
        #key.find("-1") != -1 and key.find("open-1") == -1) or
        if ( key.find("close") != -1 ):
            justlast.append(key)
    df=df.drop(columns=justlast)
    return df

def expand_previous(dataframe, window=10):
    df = dataframe.copy()
    if window >= len(df):
        for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df.loc[window:len(df),"low-"+str(i)]=None
            #df.loc[window:len(df),"open-"+str(i)]=None            
            df.loc[window:len(df),"close-"+str(i)]=None            
            df.loc[window:len(df),"volume-"+str(i)]=None
        window=len(df)

    for i in range(1, window+1):
        try:
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i].to_list()

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i].to_list()

            # df.loc[window:len(df),"open-"+str(i)]=None
            # df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i].to_list()           
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i].to_list()            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i].to_list()
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
        except:
            prerr("Error in     expand_previous: " +str(i))
    if window >= len(df): return df       
    return df.iloc[window:]

def expand_previous_org(dataframe, window=10):
    df = dataframe.copy()
    if window >= len(df):
        for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df.loc[window:len(df),"low-"+str(i)]=None
            df.loc[window:len(df),"open-"+str(i)]=None            
            df.loc[window:len(df),"close-"+str(i)]=None            
            df.loc[window:len(df),"volume-"+str(i)]=None
        window=len(df)

    for i in range(1, window+1):
        try:
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i].to_list()

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i].to_list()

            df.loc[window:len(df),"open-"+str(i)]=None
            df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i].to_list()           
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i].to_list()            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i].to_list()
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
        except:
            prerr("Error in     expand_previous: " +str(i))
    if window >= len(df): return df       
    return df.iloc[window:]


def expand_previous_err(dataframe, window=10):
    df = dataframe.copy()
    if window >= len(df):
        for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df.loc[window:len(df),"low-"+str(i)]=None
            df.loc[window:len(df),"open-"+str(i)]=None            
            df.loc[window:len(df),"close-"+str(i)]=None            
            df.loc[window:len(df),"volume-"+str(i)]=None
        window=len(df)

    for i in range(1, window+1):
            df.loc[window:len(df),"high-"+str(i)]=None
            df["high-"+str(i)].iloc[window:len(df)]=df["high"][window-i:len(df)-i]

            df.loc[window:len(df),"low-"+str(i)]=None
            df["low-"+str(i)].iloc[window:len(df)]=df["low"][window-i:len(df)-i]

            df.loc[window:len(df),"open-"+str(i)]=None
            df["open-"+str(i)].iloc[window:len(df)]=df["open"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"close-"+str(i)]=None
            df["close-"+str(i)].iloc[window:len(df)]=df["close"][window-i:len(df)-i]            
            
            df.loc[window:len(df),"volume-"+str(i)]=None
            df["volume-"+str(i)].iloc[window:len(df)]=df["volume"][window-i:len(df)-i]
            
            # df["high-"+str(i)][i:] = df["high"][i-1:]
            # df["low-"+str(i)][i:] = df["low"][i-1:]
            # df["open-"+str(i)][i:] = df["open"][i-1:]
            # df["close-"+str(i)][i:] = df["close"][i-1:]
            # df["volume-"+str(i)][i:] = df["volume"][i-1:]
    if window >= len(df):
        return df
    return df.iloc[window:]

def expand_timeframe(df_minutes,df_hours, window=2):
    dfm = df_minutes.copy()
    for j in range(1, window+1):
        for i in df_hours[dfm.iloc[0].name:].index:
        #prerr(str(i))
            try:
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"high_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['high']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"low_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['low']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"open_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['open']
                dfm.loc[pd.date_range(str(i), periods=60, freq="min"),"close_1h-"+str(j)]= df_hours[str(i-pd.Timedelta(str(j)+" hour"))]['close']
            except:
                prerr("Error Merging: "+str(i))
    
    return dfm


def float_or_not(var):
    try:
        x=float(var)
    except:
        x=None
    return x

def expand_to_1h(df_1m,df_1h, window=2):
    dfm = df_1m.copy()
    index_start=df_1h.index.intersection(dfm.index.round(freq='H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=60, freq="min"))
                dfm.loc[timefragment,"high_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['high'])
                dfm.loc[timefragment,"low_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['low'])
                dfm.loc[timefragment,"open_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['open'])
                dfm.loc[timefragment,"close_1h-"+str(j)]=float_or_not(df_1h.loc[str(i-pd.Timedelta(str(j)+" hour"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_4h(df_1m,df_4h, window=2):
    dfm = df_1m.copy()
    #index_start=df_1h[str(dfm.iloc[0].name.round(freq='H')):].index.intersection(dfm.index)
    index_start=df_4h.index.intersection(dfm.index.round(freq='4H'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=4*60, freq="min"))
                dfm.loc[timefragment,"high_4h-"+str(j)]=float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['high'])
                dfm.loc[timefragment,"low_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['low'])
                dfm.loc[timefragment,"open_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['open'])
                dfm.loc[timefragment,"close_4h-"+str(j)]= float_or_not(df_4h.loc[str(i-pd.Timedelta(str(j*4)+" hour"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_1d(df_1m,df_1d, window=2,time_suffix="1d"):
    dfm = df_1m.copy()
    index_start=df_1d.index.intersection(dfm.index.round(freq='1d'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                prerr(i)
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=24*60, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_1d.loc[str(i-pd.Timedelta(str(j)+" day"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def expand_to_5m(df_1m,df_5m, window=2,time_suffix="5m"):
    dfm = df_1m.copy()
    index_start=df_5m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):
            # try:    
                
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=5, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_5m.loc[str(i-pd.Timedelta(str(j*5)+" min"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm


def expand_to_15m(df_1m,df_15m, window=2,time_suffix="15m"):
    dfm = df_1m.copy()
    index_start=df_15m.index.intersection(dfm.index.round(freq='5 min'))
    for i in index_start:
        for j in range(1, window+1):    
            # try:    
                timefragment=dfm.index.intersection(pd.date_range(str(i), periods=15, freq="min"))
                dfm.loc[timefragment,"high_"+time_suffix+"-"+str(j)]=float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['high'])
                dfm.loc[timefragment,"low_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['low'])
                dfm.loc[timefragment,"open_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['open'])
                dfm.loc[timefragment,"close_"+time_suffix+"-"+str(j)]= float_or_not(df_15m.loc[str(i-pd.Timedelta(str(j*15)+" min"))]['close'])
            # except:
            #     prerr("error fonction "str(i))
    return dfm

def rapid1d_expand(df1m,df1d,window=2):
    d1min=df1m.copy()
    d1day=df1d.loc[
    d1min.index[0].round(freq='1d')-pd.Timedelta(str(window)+' day'):
    d1min.index[len(d1min)-1].round(freq='1d')+pd.Timedelta('1 day')
    ].copy()
    d1day_pre=expand_previous(d1day,window)
    d1day_pre=d1day_pre.drop(columns=['open', 'low','close','high','volume'])
    d1day_pre=d1day_pre.add_suffix("_day")
    d1min=pd.merge_asof(
        d1min, d1day_pre, on=None, left_on=None, right_on=None, left_index=True, 
        right_index=True, by=None, left_by=None, right_by=None, 
        suffixes=('', '_day'),
        tolerance=pd.Timedelta('1 day'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid1h_expand(df1m,df1h,window=2):
    d1min=df1m.copy()
    d1hour=df1h.loc[
    d1min.index[0].round(freq='H')-pd.Timedelta(str(window)+' hour'):
    d1min.index[len(d1min)-1].round(freq='H')+pd.Timedelta('1 hour')
    ].copy()
    d1hour_pre=expand_previous(d1hour,window)
    d1hour_pre=d1hour_pre.drop(columns=['open', 'low','close','high','volume'])
    d1hour_pre=d1hour_pre.add_suffix("_hour")
    d1min=pd.merge_asof(
    d1min, d1hour_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_hour'),
    tolerance=pd.Timedelta('1 hour'), allow_exact_matches=True, direction='backward')
    return d1min


def rapid5m_expand(df1m,df5m,window=2):
    d1min=df1m.copy()
    d5min=df5m.loc[
    d1min.index[0].round(freq='5 min')-pd.Timedelta(str(window*5+10)+' min'):
    d1min.index[len(d1min)-1].round(freq='5 min')+pd.Timedelta('5 min')
    ].copy()
    d5min_pre=expand_previous(d5min,window)
    d5min_pre=d5min_pre.drop(columns=['open', 'low','close','high','volume'])
    d5min_pre=d5min_pre.add_suffix("_5min")
    d1min=pd.merge_asof(
    d1min, d5min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_5min'),
    tolerance=pd.Timedelta('5 min'), allow_exact_matches=True, direction='backward')
    return d1min

def rapid15m_expand(df1m,df15m,window=2):
    d1min=df1m.copy()
    d15min=df15m.loc[
    d1min.index[0].round(freq='15 min')-pd.Timedelta(str(window*15+30)+' min'):
    d1min.index[len(d1min)-1].round(freq='15 min')+pd.Timedelta('15 min')
    ].copy()
    d15min_pre=expand_previous(d15min,window)
    d15min_pre=d15min_pre.drop(columns=['open', 'low','close','high','volume'])
    d15min_pre=d15min_pre.add_suffix("_15min")
    d1min=pd.merge_asof(
    d1min, d15min_pre, on=None, left_on=None, right_on=None, left_index=True, 
    right_index=True, by=None, left_by=None, right_by=None, 
    suffixes=('', '_15min'),
    tolerance=pd.Timedelta('15 min'), allow_exact_matches=True, direction='backward')
    return d1min


def full_expand(df1m,df5m,df15m,df1h,df1d,window=10):
    d1min=df1m.copy()
    d1min=expand_previous(d1min,window=window).drop(columns=["volume"])
    d1min=rapid1d_expand(d1min,df1d,window)
    d1min=rapid1h_expand(d1min,df1h,window)
    d1min=rapid15m_expand(d1min,df15m,window)
    d1min=rapid5m_expand(d1min,df5m,window)
    return d1min

def full_expand_org(df1m,df5m,df15m,df1h,df1d,window=10):
    d1min=df1m.copy()
    d1min=expand_previous(d1min,window=window)
    d1min=rapid1d_expand(d1min,df1d,window)
    d1min=rapid1h_expand(d1min,df1h,window)
    d1min=rapid15m_expand(d1min,df15m,window)
    d1min=rapid5m_expand(d1min,df5m,window)
    return d1min


def day_expand(data_full):
    ser = pd.to_datetime(pd.Series(data_full.index))
    data_full["day"]=ser.dt.isocalendar().day.values
    data_full["hour"]=ser.dt.hour.values
    data_full["minute"]=ser.dt.minute.values

# merging
def pair_btc(pair="LTC/USDT",window=2):
    Pair_Full=full_expand(df_list1m[pair],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[df_list1m[pair].iloc[0].name:
        df_list1m[pair].iloc[len(df_list1m[pair])-1].name],
        df_list5m["BTC/USDT"],df_list15m["BTC/USDT"],df_list1h["BTC/USDT"],df_list1d["BTC/USDT"],window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='outer',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged






In [ ]:
def buy_results(df,min_pourcent=BUY_PERCENT):
    mino=min_pourcent*0.01
    df["buy"]=(
        ((df["high"].shift(periods=1, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| ((
          df["high"].shift(periods=2, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| ((
          df["high"].shift(periods=3, freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)
    ).replace({False: 0, True: 1}) 
 
def buy_results_gen(df,min_pourcent=BUY_PERCENT,window=3):
    mino=min_pourcent*0.01
    codep1='df["buy"]=((('
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
    code=codep1+codep2
    print(code)
    exec(code)

def buy_sell(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    mino=buy_pourcent*0.01
    maxo=-sell_pourcent*0.01
    codep1='df["buy"]=((('
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
    code=codep1+codep2
    prerr(code)
    exec(code)
    codep1='df["sell"]=((df["buy"]==0)&(( '
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )& (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
    code=codep1+codep2
    prerr(code)
    exec(code)
    df["bs"]=((df['buy']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})

def buy_only(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=3):
    mino=buy_pourcent*0.01
    maxo=-sell_pourcent*0.01
    codep1='df["buy"]=((('
    for i in range(1,window):
        codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino )| (('
    codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] >=mino)).replace({False: 0, True: 1})'
    code=codep1+codep2
    prerr(code)
    exec(code)
    # codep1='df["sell"]=((df["buy"]==0)&(( '
    # for i in range(1,window):
    #     codep1=codep1+'df["high"].shift(periods='+str(i)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )& (('
    # codep2='df["high"].shift(periods='+str(window)+', freq=None, axis=0, fill_value=None)-df["high"])/df["high"] <=maxo )).replace({False: 0, True: 1})'
    # code=codep1+codep2
    # prerr(code)
    # exec(code)
    # df["bs"]=((df['buy']==1 ) & (df['sell']==0)).replace({False: 0, True: 1})
    return df

In [ ]:
def Meta_expand(data_full,metadt,pair):
    data_full["lunch_day"]=int(-(pd.to_datetime(metadt[metadt["Pair"] == pair]["launch_minute"])-pd.Timestamp('2020-01-01 00:00:00.000000')).dt.days)

def mini_expand(pair="LTC/USDT",i=0,j=10000,window=2):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window-1) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    return Merged

def mini_expand2(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window-1) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=7)
    return Merged

def mini_expand3(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData,high_weight=3):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='1 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" d")).round(freq='5 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list15m["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='15 min'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1h["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 H'):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list1d["BTC/USDT"],#.loc[(Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" day")).round(freq='1 d'):Pair_Full.iloc[len(Pair_Full)-1].name],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    # Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
    #         right_index=True, suffixes=('', ''))
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='inner',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    #buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    buy_only(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    Merged["high"]=(Merged["open"]+high_weight*Merged["high"]+Merged["low"]+Merged["close"])/(3+high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged["BTC_high"]=(Merged["BTC_open"]+high_weight*Merged["BTC_high"]+Merged["BTC_low"]+Merged["BTC_close"])/(3+high_weight)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    # Merged=justlast_remover(Merged)
    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["price"]-Merged[key])/Merged["price"]
    return Merged

def mini_expand3old(pair="LTC/USDT",i=0,j=10000,window=2,metadata=MetaData,high_weight=3):
    Pair_Full=full_expand(df_list1m[pair].iloc[i:j],df_list5m[pair],df_list15m[pair],df_list1h[pair],df_list1d[pair],window)
    BTC_Full=full_expand(
        df_list1m["BTC/USDT"].loc[Pair_Full.iloc[0].name-pd.Timedelta(str(window) +" min"):Pair_Full.iloc[len(Pair_Full)-1].name],
        df_list5m["BTC/USDT"],
        df_list15m["BTC/USDT"],
        df_list1h["BTC/USDT"],
        df_list1d["BTC/USDT"],
        window)   
    BTC_Full=BTC_Full.add_prefix("BTC_")
    Merged=pd.merge(Pair_Full, BTC_Full, left_index=True, how='left',
            right_index=True, suffixes=('', ''))
    day_expand(Merged)
    Meta_expand(Merged,metadata,pair)
    buy_sell(Merged,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=MAX_FORCAST_SIZE)
    Merged["high"]=(Merged["open"]+high_weight*Merged["high"]+Merged["low"]+Merged["close"])/(3+high_weight)
    Merged.rename(columns={"high":"price"},inplace = True)
    Merged["BTC_high"]=(Merged["BTC_open"]+high_weight*Merged["BTC_high"]+Merged["BTC_low"]+Merged["BTC_close"])/(3+high_weight)
    Merged.rename(columns={"BTC_high":"BTC_price"},inplace = True)
    # Merged=Merged.drop(columns=["BTC_open","BTC_low","BTC_close","open","low","close"])
    Merged=justlast_remover(Merged)

    for key in Merged.keys():
        if key.find("BTC")!=-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["BTC_price"]-Merged[key])/Merged["BTC_price"]
        if key.find("BTC")==-1 and (key.find("open")!=-1 or
        key.find("high")!=-1 or key.find("low")!=-1 or key.find("close")!=-1):
            Merged[key]=(Merged["price"]-Merged[key])/Merged["price"]
    return Merged



def slow_expand(pair="LTC/USDT",i=0,j=100000,window=3):
    df=mini_expand(pair=pair,i=i,j=j,window=window)
    for mx in range(1,int(len(df_list1m[pair])/j)+1) :
        df=pd.concat([df,
        mini_expand(pair=pair,
        i=(mx*j)-window,
        j=(mx+1)*j,
        window=window)],axis=0)
    return df

def pair_data_gen(pair="LTC/USDT",i=0,j=100000,window=3,metadata=MetaData):
    df=mini_expand3(pair=pair,i=i,j=j,window=window,metadata=metadata)
    for mx in range(1,int(len(df_list1m[pair])/j)+1) :
        df=pd.concat([df,
        mini_expand3(pair=pair,
        i=(mx*j)-window,
        j=(mx+1)*j,
        window=window,metadata=metadata)],axis=0)
        # Meta_expand(df,metadata,pair)
        # buy_sell(df,buy_pourcent=BUY_PERCENT,sell_pourcent=SELL_PERCENT,window=7)
        #print("loop "+str(mx)+"--> size of df: "+str(len(df)))
    return df

###

def data_is_enough(df,days=10,window=10):
    if days <= window:
        return df[~df.isnull().any(axis=1) |(df["open-"+str(days)+"_day"].isnull() & ~df["open-"+str(window-1)+"_hour"].isnull()  & ~df["open-"+str(window-1)+"_5min"].isnull() & ~df["open-"+str(days-1)+"_day"].isnull())]
    else:
        prerr("number of days must be equal or lower than window")
        return df

def data_cleanup(df):
    return df[~df.isnull().any(axis=1)]
    

def data_shufler(df):
    x = len(df)
    df["num_index"] = range(0, x, 1)
    df.set_index(df['num_index'], inplace=True)
    df = df.reindex(np.random.permutation(df.index))
    df= df.drop("num_index",axis=1)
    df = df.reindex(np.random.permutation(df.index))
    return df
    
def data_np_shufler(df):
    df = shuffle(df)
    #df = df.reindex(np.random.permutation(df.index))
        



def data_chooser(df,weight=50,row_numbers=100000):
    df=data_shufler(df)
    if row_numbers>=len(df):
        row_numbers=len(df)
    df=pd.concat([df[df["buy"]==1].iloc[:int(row_numbers*weight*0.01)],
                 df[df["buy"]==0].iloc[:int(row_numbers*(100-weight)*0.01)]])
    df=data_shufler(df)
    return df
    

def data_looper(pair_list=Binance_USDT_HALAL,window=15,buy_weight=50,sample_size=100000,min_days=10,buffer_size=100000):
    xdf=pd.DataFrame()
    for pair in pair_list:
        if pair != "BTC/USDT":
            print("working on: "+pair)
            df=pair_data_gen(pair=pair,i=0,j=buffer_size,window=window)
            gc.collect()

            df=data_is_enough(df,days=min_days,window=window)
            gc.collect()

            df=data_chooser(df,weight=buy_weight,row_numbers=sample_size)
            gc.collect()

            print(pair+" is processed")
            xdf=pd.concat([xdf,df],axis=0)
            del(df)
            gc.collect()
        else:
            print("ignore BTC")
    return xdf

def data_looper_fast(pair_list=Binance_USDT_HALAL,window=15,buy_weight=50,sample_size=100000,min_days=10,buffer_size=100000):
    xdf=pd.DataFrame()
    count=0
    for pair in pair_list:
        if pair != "BTC/USDT":
            print("working on: "+pair ,end=" -->")
            try:
                df=pair_data_gen(pair=pair,i=0,j=buffer_size,window=window)
                gc.collect()
                count+=1
                # df=data_is_enough(df,days=min_days,window=window)
                # gc.collect()
                df.reindex(np.random.permutation(df.index))
                df=data_chooser(df,weight=buy_weight,row_numbers=sample_size)
                gc.collect()

                df=data_cleanup(df)
                print(pair+" is processed")
            except:
                print(f"error while processing {pair} {count}/{len(pair_list)}")
            xdf=pd.concat([xdf,df],axis=0)
            del(df)
            gc.collect()
        else:
            print("ignore BTC")
            
    return xdf

def volume_cleaner(df):
    VolRemover=["volume","volume-1","BTC_volume","BTC_volume-1"]
    for key in df.keys():
        if key.find("volume-1_") != -1 :
            VolRemover.append(key)
    df=df.drop(columns=VolRemover)
    return df
    
Normalization=None
def normalize(dataset,file=Normalization_File):
    global Normalization
    try:
        N=Normalization
    except:
        Normalization=None
    if(Normalization==None):
        #print('Loading normalization from file')
        with open(file) as json_file:
            Normalization = json.load(json_file)
    else:
        #print('normalization is loaded')
        pass

    mean=np.array(Normalization["mean"])
    std=np.array(Normalization["std"])
    dataset -= mean 
    dataset /= std
    return(dataset)

# Start Data V2 genration Process

# Manual Data

In [ ]:
if  not JUST_IMPORT_DATA:
        pair_list=Binance_USDT_HALAL
        window=WINDOW_SIZE
        buy_weight=50
        sample_size=100000
        min_days=10
        buffer_size=100000
        xdf=pd.DataFrame()
        count=0
        for pair in pair_list:
                if pair != "BTC/USDT":
                        print("working on: "+pair ,end=" -->")
                        try:
                                df=mini_expand3(pair=pair,
                                i=0,
                                j=len(df_list1m[pair]),
                                window=5,metadata=MetaData)
                                gc.collect()
                                count+=1
                                gc.collect()
                                df=data_cleanup(df)
                                df=df.reset_index().drop(columns="date")
                                print(pair+f" is processed  {count}/{len(pair_list)}")
                        except:
                                print(f"error while processing {pair} {count}/{len(pair_list)}")
                        xdf=pd.concat([xdf,df],axis=0)
                        del(df)
        # pair="ENS/USDT"
        # mini_expand3(pair=pair,
        #         i=0,
        #         j=len(df_list1m[pair]),
        #         window=5,metadata=MetaData)
        #xdf.reindex(np.random.permutation(xdf.index))
        #xdf.reindex(np.random.permutation(xdf.index))
        xdf=xdf.reset_index()
        gc.collect()
        xdf=xdf.drop(columns="index")
        gc.collect()

        xdf=xdf[~xdf.isna().any(axis=1)]
        gc.collect()

        xdf = xdf.reindex(np.random.permutation(xdf.index))
        gc.collect()

        print(xdf.shape)


In [ ]:
if  not JUST_IMPORT_DATA:
    print("saveing the file")
    xdf.to_csv(REMOTE_DATA_FILE)
    # dffull=df
    print(xdf)

In [ ]:
dt=xdf.to_numpy()

In [3]:
if JUST_IMPORT_DATA:
    df=pd.read_csv(REMOTE_DATA_FILE,header=0,skiprows=SKIPROWS,nrows=NROWS,index_col=0)
    # df=dffull

In [4]:
df.iloc[:33]

,price,high-1,low-1,close-1,volume-1,high-2,low-2,close-2,volume-2,high-3,...,BTC_volume-4_5min,BTC_high-5_5min,BTC_low-5_5min,BTC_close-5_5min,BTC_volume-5_5min,day,hour,minute,lunch_day,buy
0,1.970000,-3.553299e-03,6.091371e-03,0.000000e+00,8647.600,0.004061,0.008629,0.004061,11332.500,0.005076,...,205.951570,-0.001932,0.002158,0.002158,189.686450,7,7,32,-638,1
1,47.533333,-7.924264e-03,-3.927069e-03,-3.927069e-03,26.880,-0.007924,-0.007924,-0.007924,4.040,-0.012763,...,368.071000,-0.015017,-0.013191,-0.013671,145.428490,2,8,30,-231,0
2,0.204567,-3.584813e-03,3.258921e-04,-1.629461e-04,139662.000,-0.004562,0.000815,-0.002118,507211.000,0.000326,...,1392.967100,-0.001305,0.005172,0.001789,674.880880,5,16,58,580,1
3,3.119833,-3.258721e-03,9.081682e-04,-1.335541e-03,4790.525,-0.003900,-0.000374,-0.001336,1901.970,-0.004220,...,390.089177,-0.007419,-0.002968,-0.006319,301.487249,3,23,30,653,1
4,3.452033,-6.711021e-03,-4.538388e-04,-3.553461e-03,7052.850,-0.011780,-0.006827,-0.007783,6833.620,-0.012881,...,119.648721,-0.014259,-0.011188,-0.013173,286.144978,7,14,8,-287,1
5,0.003335,-5.397301e-03,1.300392e-16,1.300392e-16,17710.000,-0.000300,-0.000300,-0.000300,0.000,-0.000300,...,156.361561,0.003156,0.004930,0.003334,93.192070,1,22,50,-187,1
6,0.218850,-5.757368e-03,-4.569340e-04,-4.569340e-04,243133.300,-0.006626,-0.004021,-0.005757,294771.400,-0.004112,...,119.605800,0.001362,0.003225,0.002545,138.725477,1,10,1,316,1
7,0.353233,-6.983108e-03,5.473247e-03,6.605643e-04,16437.000,-0.003303,0.007172,0.005473,47217.000,-0.010380,...,94.478330,-0.018087,-0.014353,-0.014554,131.290550,5,21,36,98,1
8,16.071167,-5.776390e-03,2.188184e-03,-3.629689e-04,34697.252,-0.007892,-0.005154,-0.005652,7356.852,-0.008079,...,232.505711,-0.008467,-0.001697,-0.005604,555.361578,3,8,42,-230,1
9,6.097500,-4.100041e-04,-8.200082e-05,-4.100041e-04,829.940,-0.000410,0.000738,-0.000410,2869.890,-0.000082,...,263.364620,0.005490,0.007673,0.005490,278.844350,1,15,39,-800,0


In [5]:
#dffull=df
size_data=sys.getsizeof(df)/(1024**2)
print(size_data)
#if size_data >= 18000:
#    df=df[~df.isna().any(axis=1)].iloc[0:int(len(dffull/2))]
#df=dffull.drop(columns=["sell",'buy'])

6347.656265258789


In [6]:
buy=df.pop(df.columns[-1])
target=buy

In [6]:
target[0:100]

1010824
2717856     0
11514729    0
1752644     0
7267540     0
13518888    0
           ..
3504445     0
2921113     0
12909756    0
4853877     0
7565172     0
Name: 0, Length: 100, dtype: int64

In [ ]:
580278/14614047 * 100

# Part 2: Training


## Testing impoted DATA

### Remove volumes

Cheking null values

In [ ]:
#df[df.isnull().any(axis=1)]
#dffull=df



checking keys

In [ ]:
# for k in df.keys():
#     if k.find("-1") != -1 :print(k ,end="|")
gc.collect()

# starting numpy process

## convert Pandas DataFrame to numpy object

In [7]:
index_20percent= int(0.25*len(df.iloc[:,0]))
print(index_20percent)

1000000


In [8]:
XVALIDATION= df.iloc[:index_20percent]
YVALIDATION= buy.iloc[:index_20percent]


In [9]:
XTRAIN= df.iloc[index_20percent:]
YTRAIN= buy.iloc[index_20percent:]

## Normalization

In [10]:
import gc
gc.collect()

80

Tenderalization (mean normalization)

In [11]:
# Normalization
if TESTING_MOD:
    with open(Normalization_File) as json_file:
                print(Normalization_File)
                Normalization = json.load(json_file)
                mean=np.array(Normalization["mean"])
                std=np.array(Normalization["std"])
# mean=np.array(Normalization["mean"])


/UltimeTradingBot/Data/tp28_w5_max1min_Norm_v1.json


In [ ]:
gc.collect()

if not TESTING_MOD:
    if FIRST_NORM_FLAG:
        print("normalizing ...")
        mean = XTRAIN.mean(axis=0)
        std = XTRAIN.std(axis=0)



        XTRAIN -= mean 
        XTRAIN /= std

        XVALIDATION -=mean
        XVALIDATION /= std
        FIRST_NORM_FLAG=False
        ######################### SAVIN NORM ################
        try:
            Normalization={"mean":mean.tolist(),"std":std.tolist()}
            with open(Normalization_File, 'w+') as fp:
                        json.dump(Normalization, fp,  indent=4)
                        print(Normalization_File)
        except:
            print("error Normalization in juppiter")
    else:print("already normalized")



In [12]:
################## V2 Testing ######################
if  TESTING_MOD:
    if  FIRST_NORM_FLAG:
        print("normalizing ...")
        XTRAIN -= Normalization["mean"]
        XTRAIN /=  Normalization["std"]

        XVALIDATION -= Normalization["mean"]
        XVALIDATION /=  Normalization["std"]

    
        FIRST_NORM_FLAG=False
        FIRST_NORM_FLAG=False
    else:print("already normalized")


normalizing ...


In [14]:
print(XTRAIN.shape)
print(YTRAIN.shape)
print(XVALIDATION.shape)
print(YVALIDATION.shape)

(3000000, 206)
(3000000,)
(1000000, 206)
(1000000,)


In [ ]:
df=pd.concat([XTRAIN,XVALIDATION])

# Neural network model

## Model Test

## Compile the model 

In [ ]:
if not TESTING_MOD:
    IN_DIM=len(mean)
#     model = Sequential()
#     model.add(Dense(int(IN_DIM/2),input_dim=IN_DIM,activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(int(IN_DIM/4),activation='relu'))
# #    model.add(Dropout(0.7))
#     model.add(Dense(8,activation='relu'))
# #    model.add(Dropout(0.7))
#     model.add(Dense(1,activation='sigmoid'))
    model = Sequential()
    model.add(Dense(50,input_dim=IN_DIM,activation='relu'))
    model.add(Dense(10,activation='relu'))
    model.add(Dense(10,activation='relu'))

#    model.add(Dropout(0.5))
#    model.add(Dense(20,activation='relu'))
#    model.add(Dropout(0.7))
    model.add(Dense(4,activation='relu'))
#    model.add(Dropout(0.7))
    model.add(Dense(1,activation='sigmoid'))
    print(model.summary())
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    callbacks_a = ModelCheckpoint(filepath=Model_FileName+'zero.hdf5',monitor ='val_accuracy',save_best_only = True, save_weights = True)
    callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=10,verbose=1)
    history = model.fit(XTRAIN,
                    YTRAIN,
                    validation_data=(XVALIDATION,YVALIDATION),
                    epochs=6000,
                    batch_size=128*5,
                    callbacks=[callbacks_a,callbacks_b])

    print('##########################################################################')
    print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
    #94%

In [ ]:
if not TESTING_MOD:
    IN_DIM=len(mean)
    model = Sequential()
    model.add(Dense(int(IN_DIM/2),input_dim=IN_DIM,activation='softplus'))
    model.add(Dense(int(IN_DIM/2),activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(int(IN_DIM/5),activation='softplus'))
    model.add(Dense(int(IN_DIM/4),activation='softmax'))
    model.add(Dense(int(IN_DIM/1),activation='softplus'))
    model.add(Dense(1,activation='sigmoid'))
    print(model.summary())
    #model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    callbacks_a = ModelCheckpoint(filepath=Model_FileName+'.sftp.hdf5',monitor ='val_accuracy',save_best_only = True, save_weights = True)
    callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=20,verbose=1)
    history = model.fit(XTRAIN,
                    YTRAIN,
                    validation_data=(XVALIDATION,YVALIDATION),
                    epochs=6000,
                    batch_size=128*5,
                    callbacks=[callbacks_a,callbacks_b])

    print('##########################################################################')
    print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")
    #94%

# Testing the model with the new DATA

## Loding the model

In [13]:
from keras.models import load_model

model = load_model(Model_FileName+'zero.hdf5')

print(Model_FileName+' Loaded')

/UltimeTradingBot/Data/tp28_w5_max1min_Model_v1.hdf5 Loaded


2022-10-07 13:58:20.055298: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## testing the Accuracy

In [14]:
# fixing values type

#dt=tf.convert_to_tensor(dt, dtype=tf.float32)
XTRAIN[:22]

,price,high-1,low-1,close-1,volume-1,high-2,low-2,close-2,volume-2,high-3,...,BTC_close-4_5min,BTC_volume-4_5min,BTC_high-5_5min,BTC_low-5_5min,BTC_close-5_5min,BTC_volume-5_5min,day,hour,minute,lunch_day
1000000,-0.180864,-0.079175,0.715293,-0.353903,-0.046719,-0.222931,-0.426748,-0.110995,-0.044225,-0.219713,...,-0.414964,-0.382215,-0.485494,-0.576012,-0.382593,-0.311422,0.029675,0.070964,-1.151031,0.819145
1000001,-0.127188,1.046094,0.201800,0.930976,-0.046723,0.811146,0.331497,0.724925,-0.044226,0.680173,...,-0.020370,-0.604374,0.044813,-0.210628,-0.080555,-0.544173,1.544382,-0.654953,-0.920965,-0.270866
1000002,-0.180847,-0.616071,1.087354,2.309861,-0.046716,-0.461518,-1.213976,-0.978869,-0.044226,-0.422988,...,0.230371,-0.119685,-0.350929,0.202571,0.103905,0.338478,-0.475227,0.216148,0.344398,-0.967758
1000003,-0.099081,0.653015,-0.074035,0.075245,-0.046723,0.809999,0.330103,0.723389,-0.044226,0.615299,...,0.132627,-0.399935,0.143042,-0.223035,-0.027139,-0.431102,1.039480,0.942064,0.977079,-0.623779
1000004,-0.183772,-0.087481,-0.140712,0.397657,-0.046602,-0.737162,-0.929049,-0.664753,-0.044212,-0.784687,...,-0.088830,-0.499746,0.047232,-0.229766,-0.093141,-0.408132,1.039480,-0.219403,0.229365,0.160225
1000005,0.356850,0.481703,-0.051060,0.437058,-0.046723,0.285040,0.020694,0.193576,-0.044226,0.311797,...,-0.273809,0.492325,-0.585233,-0.404860,-0.412802,0.570604,1.039480,0.361331,-0.288283,2.195061
1000006,-0.183543,0.518124,0.421112,0.730327,-0.046706,0.526069,1.519307,0.821899,-0.044217,1.554625,...,2.748015,3.085732,0.697277,1.020262,0.699483,0.415421,-0.980130,0.070964,-0.633382,0.023973
1000007,-0.178384,-3.550794,0.400266,1.240005,-0.046286,-2.006273,-1.099990,-1.323289,-0.043885,-2.120651,...,-5.148639,5.108826,-8.678203,-8.399736,-9.423529,0.009299,1.039480,-0.945319,-0.920965,1.915857
1000008,-0.181615,-0.254048,0.550526,-0.747537,-0.046722,-0.531583,-0.033986,0.322002,-0.044225,-0.612453,...,-1.570171,-0.226968,-1.203917,-1.247865,-1.496179,-0.020147,-0.475227,0.361331,0.747013,-1.070505
1000009,-0.176109,0.006721,-0.950173,-0.862744,-0.046723,-0.149616,-0.534754,-0.230066,-0.044226,-0.560668,...,0.354399,-0.389706,-0.121198,0.196596,0.383509,-0.100805,0.534577,0.361331,0.229365,-1.171019


In [37]:
# XTRAIN -= Normalization["mean"]
# XTRAIN /=  Normalization["std"]


In [21]:
accuracy = model.evaluate(XTRAIN[YTRAIN==0], YTRAIN[YTRAIN==0])

49982/49982 [==============================] - 60s 1ms/step - loss: 0.0696 - accuracy: 0.9713


# Continue The Training

In [19]:
if RETRAIN:
    callbacks_a = ModelCheckpoint(filepath=Model_FileName+'50-50Plus.hdf5',monitor ='val_accuracy',save_best_only = True, save_weights = True)
    callbacks_b = EarlyStopping(monitor ='val_accuracy',mode='auto',patience=20,verbose=1)
    history = model.fit(XVALIDATION,YVALIDATION,
                        validation_data=(XTRAIN,
                        YTRAIN),
                        epochs=6000,
                        batch_size=128*5,
                        callbacks=[callbacks_a,callbacks_b])

Epoch 1/6000
1563/1563 [==============================] - 22s 14ms/step - loss: 0.0583 - accuracy: 0.9797 - val_loss: 0.0562 - val_accuracy: 0.9805
Epoch 2/6000
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0561 - accuracy: 0.9806 - val_loss: 0.0557 - val_accuracy: 0.9808
Epoch 3/6000
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0555 - accuracy: 0.9808 - val_loss: 0.0555 - val_accuracy: 0.9809
Epoch 4/6000
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0552 - accuracy: 0.9809 - val_loss: 0.0556 - val_accuracy: 0.9808
Epoch 5/6000
1563/1563 [==============================] - 15s 9ms/step - loss: 0.0549 - accuracy: 0.9809 - val_loss: 0.0556 - val_accuracy: 0.9808
Epoch 6/6000
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0547 - accuracy: 0.9810 - val_loss: 0.0555 - val_accuracy: 0.9808
Epoch 7/6000
1563/1563 [==============================] - 12s 8ms/step - loss: 0.0544 - accuracy: 0.9812 - val_loss: 

In [ ]:
print('##########################################################################')
print(f"------val_accuracy-----> {'{0:.4g}'.format(max(history.history['val_accuracy'])*100)} | {'{0:.4g}'.format(max(history.history['accuracy'])*100)} <----------accuracy----------")


### Use the model

In [ ]:

# make probability predictions with the model
predictions = model.predict(dt[0:,0:-1])
# round predictions     
rounded = [round(x[0]) for x in predictions]

In [25]:
df.columns

Index(['price', 'high-1', 'low-1', 'close-1', 'volume-1', 'high-2', 'low-2',
       'close-2', 'volume-2', 'high-3',
       ...
       'BTC_close-4_5min', 'BTC_volume-4_5min', 'BTC_high-5_5min',
       'BTC_low-5_5min', 'BTC_close-5_5min', 'BTC_volume-5_5min', 'day',
       'hour', 'minute', 'lunch_day'],
      dtype='object', length=206)

In [ ]:
sum(rounded)/len(rounded)

In [ ]:
YVALIDATION[1002]

In [ ]:
def Buy_Dessision(input):
    predictions = model.predict(XVALIDATION)
    rounded = [round(x[0]) for x in predictions]